In [ ]:
!curl -O http://ufldl.stanford.edu/housenumbers/train_32x32.mat

In [ ]:
!curl -O http://ufldl.stanford.edu/housenumbers/test_32x32.mat

In [7]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from scipy import io

In [8]:
train_data = io.loadmat('train_32x32.mat')

In [9]:
test_data = io.loadmat('test_32x32.mat')

In [10]:
train_X = train_data['X']
train_Y= train_data['y']